In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import datetime


In [2]:
# Load the data from gas.csv
df = pd.read_csv('gas1.csv')

In [3]:
df

,Location,Month,Prices
0,"Charlottetown and Summerside, Prince Edward Is...",Jan-90,50.3
1,"Charlottetown and Summerside, Prince Edward Is...",Feb-90,50.5
2,"Charlottetown and Summerside, Prince Edward Is...",Mar-90,50.8
3,"Charlottetown and Summerside, Prince Edward Is...",Apr-90,51.1
4,"Charlottetown and Summerside, Prince Edward Is...",May-90,51.7
...,...,...,...
6947,"Yellowknife, Northwest Territories",Aug-23,175.7
6948,"Yellowknife, Northwest Territories",Sep-23,180.3
6949,"Yellowknife, Northwest Territories",Oct-23,183.1
6950,"Yellowknife, Northwest Territories",Nov-23,187.3


In [4]:
# Convert 'Month' column to datetime format
df['Month'] = pd.to_datetime(df['Month'], format='%b-%y')


In [5]:
# Extract year and month as separate numerical features
df['Year'] = df['Month'].dt.year
df['Month_Num'] = df['Month'].dt.month

In [6]:
# Clean the data by removing rows with non-numeric values in 'prices' column
df = df[pd.to_numeric(df['Prices'], errors='coerce').notnull()]


In [7]:
# Convert 'prices' column to float
df['Prices'] = df['Prices'].astype(float)


C:\Users\User\AppData\Local\Temp\ipykernel_15752\497285554.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Prices'] = df['Prices'].astype(float)


In [8]:
# Group the data by 'Location' for predictions
groups = df.groupby('Location')
df

,Location,Month,Prices,Year,Month_Num
0,"Charlottetown and Summerside, Prince Edward Is...",1990-01-01,50.3,1990.0,1.0
1,"Charlottetown and Summerside, Prince Edward Is...",1990-02-01,50.5,1990.0,2.0
2,"Charlottetown and Summerside, Prince Edward Is...",1990-03-01,50.8,1990.0,3.0
3,"Charlottetown and Summerside, Prince Edward Is...",1990-04-01,51.1,1990.0,4.0
4,"Charlottetown and Summerside, Prince Edward Is...",1990-05-01,51.7,1990.0,5.0
...,...,...,...,...,...
6947,"Yellowknife, Northwest Territories",2023-08-01,175.7,2023.0,8.0
6948,"Yellowknife, Northwest Territories",2023-09-01,180.3,2023.0,9.0
6949,"Yellowknife, Northwest Territories",2023-10-01,183.1,2023.0,10.0
6950,"Yellowknife, Northwest Territories",2023-11-01,187.3,2023.0,11.0


In [9]:
# Initialize an empty DataFrame to store the predictions
predicted_df = pd.DataFrame()
predicted_df

""


In [10]:
# Iterate through each group and make predictions
for name, group in groups:
    # Splitting the data into features (X) and target variable (y)
    X = group[['Year', 'Month_Num']]
    y = group['Prices']

    # Training a linear regression model
    model = LinearRegression()
    model.fit(X, y)

    # Create a DataFrame for future dates for this location
    future_dates = pd.date_range(start=group['Month'].max(), periods=60, freq='M') + datetime.timedelta(days=30)
    future_df = pd.DataFrame({'Month': future_dates})
    future_df['Year'] = future_df['Month'].dt.year
    future_df['Month_Num'] = future_df['Month'].dt.month

    # Make predictions for this location
    future_predictions = model.predict(future_df[['Year', 'Month_Num']])

    # Add predictions to the predicted DataFrame
    future_df['Predicted Prices'] = future_predictions
    future_df['Location'] = name

    # Append the predictions for this location to the overall predicted DataFrame
    predicted_df = pd.concat([predicted_df, future_df])


In [11]:
predicted_df

,Month,Year,Month_Num,Predicted Prices,Location
0,2024-01-30,2024,1,140.012349,"Calgary, Alberta"
1,2024-03-01,2024,3,140.904472,"Calgary, Alberta"
2,2024-03-30,2024,3,140.904472,"Calgary, Alberta"
3,2024-04-30,2024,4,141.350533,"Calgary, Alberta"
4,2024-05-30,2024,5,141.796594,"Calgary, Alberta"
...,...,...,...,...,...
55,2028-08-30,2028,8,174.496013,"Yellowknife, Northwest Territories"
56,2028-09-30,2028,9,174.964154,"Yellowknife, Northwest Territories"
57,2028-10-30,2028,10,175.432294,"Yellowknife, Northwest Territories"
58,2028-11-30,2028,11,175.900435,"Yellowknife, Northwest Territories"


In [33]:
# Reset the index of the predicted DataFrame
predicted_df.reset_index(drop=True, inplace=True)
predicted_df

,Month,Year,Month_Num,Predicted Prices,Location
0,2024-01-30,2024,1,140.012349,"Calgary, Alberta"
1,2024-03-01,2024,3,140.904472,"Calgary, Alberta"
2,2024-03-30,2024,3,140.904472,"Calgary, Alberta"
3,2024-04-30,2024,4,141.350533,"Calgary, Alberta"
4,2024-05-30,2024,5,141.796594,"Calgary, Alberta"
...,...,...,...,...,...
1015,2028-08-30,2028,8,174.496013,"Yellowknife, Northwest Territories"
1016,2028-09-30,2028,9,174.964154,"Yellowknife, Northwest Territories"
1017,2028-10-30,2028,10,175.432294,"Yellowknife, Northwest Territories"
1018,2028-11-30,2028,11,175.900435,"Yellowknife, Northwest Territories"


In [34]:
import hvplot.pandas
# Plotting using hvplot
plot = predicted_df.hvplot.line(x='Month', y='Predicted Prices', by='Location', xlabel='Month', ylabel='Price (Cents per Liter)', title='Diesel Prices Prediction')#, hover_cols=['Location'])

# Show the plot
plot

:NdOverlay   [Location]
   :Curve   [Month]   (Predicted Prices)

In [21]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import datetime  


In [22]:
# Load the data from gas.csv
df = pd.read_csv('gas1.csv')


In [23]:
# Convert 'Month' column to datetime format
df['Month'] = pd.to_datetime(df['Month'], format='%b-%y')

# Extract year and month as separate numerical features
df['Year'] = df['Month'].dt.year
df['Month_Num'] = df['Month'].dt.month

# Clean the data by removing rows with non-numeric values in 'Prices' column
df = df[pd.to_numeric(df['Prices'], errors='coerce').notnull()]


In [24]:
# Convert 'Prices' column to float
df['Prices'] = df['Prices'].astype(float)

# Group the data by 'Location' for predictions
groups = df.groupby('Location')


In [25]:
# Initialize an empty DataFrame to store the predictions
predicted_df = pd.DataFrame()


In [26]:
# Iterate through each group and make predictions
for name, group in groups:
    # Splitting the data into features (X) and target variable (y)
    X = group[['Year', 'Month_Num']]
    y = group['Prices']

    # Training a linear regression model
    model = LinearRegression()
    model.fit(X, y)

    # Create a DataFrame for future dates for this location
    future_dates = pd.date_range(start=group['Month'].max(), periods=60, freq='M') + datetime.timedelta(days=30)
    future_df = pd.DataFrame({'Month': future_dates})
    future_df['Year'] = future_df['Month'].dt.year
    future_df['Month_Num'] = future_df['Month'].dt.month

    # Make predictions for this location
    future_predictions = model.predict(future_df[['Year', 'Month_Num']])

    # Add predictions to the predicted DataFrame
    future_df['Predicted Prices'] = future_predictions
    future_df['Location'] = name

    # Append the predictions for this location to the overall predicted DataFrame
    predicted_df = pd.concat([predicted_df, future_df])


In [27]:
# Reset the index of the predicted DataFrame
predicted_df.reset_index(drop=True, inplace=True)


In [30]:
# Merge the predicted DataFrame with the actual data to calculate percentage change
merged_df = pd.merge(df, predicted_df, on=['Location', 'Year', 'Month_Num'], how='outer')
merged_df['Price Difference'] = merged_df['Predicted Prices'] - merged_df['Prices']
merged_df['Percentage Change'] = (merged_df['Price Difference'] / merged_df['Prices']) * 100
merged_df

,Location,Month_x,Prices,Year,Month_Num,Month_y,Predicted Prices,Price Difference,Percentage Change
0,"Charlottetown and Summerside, Prince Edward Is...",1990-01-01,50.3,1990.0,1.0,NaT,NaN,NaN,NaN
1,"Charlottetown and Summerside, Prince Edward Is...",1990-02-01,50.5,1990.0,2.0,NaT,NaN,NaN,NaN
2,"Charlottetown and Summerside, Prince Edward Is...",1990-03-01,50.8,1990.0,3.0,NaT,NaN,NaN,NaN
3,"Charlottetown and Summerside, Prince Edward Is...",1990-04-01,51.1,1990.0,4.0,NaT,NaN,NaN,NaN
4,"Charlottetown and Summerside, Prince Edward Is...",1990-05-01,51.7,1990.0,5.0,NaT,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
7111,"Yellowknife, Northwest Territories",NaT,NaN,2028.0,8.0,2028-08-30,174.496013,NaN,NaN
7112,"Yellowknife, Northwest Territories",NaT,NaN,2028.0,9.0,2028-09-30,174.964154,NaN,NaN
7113,"Yellowknife, Northwest Territories",NaT,NaN,2028.0,10.0,2028-10-30,175.432294,NaN,NaN
7114,"Yellowknife, Northwest Territories",NaT,NaN,2028.0,11.0,2028-11-30,175.900435,NaN,NaN


In [32]:
# Print the merged DataFrame showing percentage change by location
merged_df[['Location', 'Month_y', 'Prices', 'Predicted Prices', 'Percentage Change']]

,Location,Month_y,Prices,Predicted Prices,Percentage Change
0,"Charlottetown and Summerside, Prince Edward Is...",NaT,50.3,NaN,NaN
1,"Charlottetown and Summerside, Prince Edward Is...",NaT,50.5,NaN,NaN
2,"Charlottetown and Summerside, Prince Edward Is...",NaT,50.8,NaN,NaN
3,"Charlottetown and Summerside, Prince Edward Is...",NaT,51.1,NaN,NaN
4,"Charlottetown and Summerside, Prince Edward Is...",NaT,51.7,NaN,NaN
...,...,...,...,...,...
7111,"Yellowknife, Northwest Territories",2028-08-30,NaN,174.496013,NaN
7112,"Yellowknife, Northwest Territories",2028-09-30,NaN,174.964154,NaN
7113,"Yellowknife, Northwest Territories",2028-10-30,NaN,175.432294,NaN
7114,"Yellowknife, Northwest Territories",2028-11-30,NaN,175.900435,NaN
